In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt

In [2]:
sensor_names=[f's_{i}' for i in range(1,22)]
settings=['set_1','set_2','set_3']
cols=['unit_nr','time_cycles']+settings+sensor_names
train=pd.read_csv(('CMAPSSData/train_FD001.txt'),sep='\s+',header=None,names=cols) # change it here
test=pd.read_csv(('CMAPSSData/test_FD001.txt'),sep='\s+',header=None,names=cols)#change it here
actual=pd.read_csv(('CMAPSSData/RUL_FD001.txt'),sep='\s+',header=None,names=['RUL'])#change it here
actual=actual.to_numpy().flatten().tolist()

In [3]:
train.loc[train['unit_nr']==1]

,unit_nr,time_cycles,set_1,set_2,set_3,s_1,s_2,s_3,s_4,s_5,...,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1,188,-0.0067,0.0003,100.0,518.67,643.75,1602.38,1422.78,14.62,...,519.79,2388.23,8117.69,8.5207,0.03,396,2388,100.0,38.51,22.9588
188,1,189,-0.0006,0.0002,100.0,518.67,644.18,1596.17,1428.01,14.62,...,519.58,2388.33,8117.51,8.5183,0.03,395,2388,100.0,38.48,23.1127
189,1,190,-0.0027,0.0001,100.0,518.67,643.64,1599.22,1425.95,14.62,...,520.04,2388.35,8112.58,8.5223,0.03,398,2388,100.0,38.49,23.0675
190,1,191,-0.0000,-0.0004,100.0,518.67,643.34,1602.36,1425.77,14.62,...,519.57,2388.30,8114.61,8.5174,0.03,394,2388,100.0,38.45,23.1295


In [4]:
test

,unit_nr,time_cycles,set_1,set_2,set_3,s_1,s_2,s_3,s_4,s_5,...,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,100,194,0.0049,0.0000,100.0,518.67,643.24,1599.45,1415.79,14.62,...,520.69,2388.00,8213.28,8.4715,0.03,394,2388,100.0,38.65,23.1974
13092,100,195,-0.0011,-0.0001,100.0,518.67,643.22,1595.69,1422.05,14.62,...,521.05,2388.09,8210.85,8.4512,0.03,395,2388,100.0,38.57,23.2771
13093,100,196,-0.0006,-0.0003,100.0,518.67,643.44,1593.15,1406.82,14.62,...,521.18,2388.04,8217.24,8.4569,0.03,395,2388,100.0,38.62,23.2051
13094,100,197,-0.0038,0.0001,100.0,518.67,643.26,1594.99,1419.36,14.62,...,521.33,2388.08,8220.48,8.4711,0.03,395,2388,100.0,38.66,23.2699


In [5]:
max_cycle_train=[]
for i in range(1,101):
    row=train.loc[train['unit_nr']==i]
    max_cycle_train.append(row['time_cycles'].max())

max_cycle_test=[]
for i in range(100):
    row=test.loc[test['unit_nr']==i+1]['time_cycles']
    max_cycle_test.append(len(row)+int(actual[i]))

In [6]:
def add_rul(data,max_cycle):
    RUL=[]
    for i,j in enumerate(max_cycle):
        row=data.loc[data['unit_nr']==i+1]['time_cycles']
        for k in row:
            RUL.append(j-k)
    return RUL

train_rul=pd.DataFrame(add_rul(train,max_cycle_train))
train['RUL']=train_rul.clip(upper=130)
test['RUL']=(add_rul(test,max_cycle_test))

In [7]:
drop_sensors = ['s_1','s_5','s_6','s_10','s_16','s_18','s_19']
index_names=['time_cycles']
drop_labels = index_names+settings+drop_sensors
X_train = train.drop(drop_labels, axis=1)
X_test = test.drop(drop_labels, axis=1)

In [8]:
col_to_scale=X_train.columns.to_list()
col_to_scale.pop(0)
col_to_scale.pop(-1)

'RUL'

In [9]:
col_to_scale

['s_2',
 's_3',
 's_4',
 's_7',
 's_8',
 's_9',
 's_11',
 's_12',
 's_13',
 's_14',
 's_15',
 's_17',
 's_20',
 's_21']

In [10]:
def exponential_smoothing(df, sensors, alpha=0.4):
    cf=[]
    for i in sensors:
        C=df[i].copy()
        X=[C[0]]
        for i in range(1,len(C)):
            X.append(alpha * C[i-1]+(1-alpha)*X[i-1])
        cf.append(X)
    return np.array(cf)
        
e_train=exponential_smoothing(X_train,col_to_scale,0.2)
e_test=exponential_smoothing(X_test,col_to_scale,0.2)
ex_train=X_train.copy()
ex_test=X_test.copy()
for i,j in enumerate(col_to_scale):
    ex_train[j]=e_train[i]
for i,j in enumerate(col_to_scale):
    ex_test[j]=e_test[i]

In [11]:
scaler=MinMaxScaler()
def scaling(x_scale):
    x_scale[col_to_scale]=scaler.fit_transform(x_scale[col_to_scale])
    return x_scale
ex_train=scaling(ex_train)
ex_test=scaling(ex_test)

In [12]:
rbm_train=ex_train[col_to_scale].to_numpy()
rbm_test=ex_test[col_to_scale].to_numpy()

In [13]:
import numpy as np
import math

class GaussianRBM:
    def __init__(self, num_visible, num_hidden):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.weights = np.random.randn(num_visible, num_hidden)
        self.visible_bias = np.zeros(num_visible)
        self.hidden_bias = np.zeros(num_hidden)

    def sigmoid(self, x):
        m=[]
        shape=x.shape
        x=x.flatten()
        for i in x:
            m.append(1.0 / (1.0 + math.exp(-i)))
        return np.array(m).reshape(shape)

    def sample_hidden_given_visible(self, visible_data):
        hidden_activations = np.dot(visible_data, self.weights) + self.hidden_bias
        hidden_samples = np.random.normal(loc=self.sigmoid(hidden_activations), scale=1)
        return hidden_samples

    def sample_visible_given_hidden(self, hidden_data):
        visible_activations = np.dot(hidden_data, self.weights.T) + self.visible_bias
        visible_samples = np.random.normal(loc=self.sigmoid(visible_activations), scale=1)
        return visible_samples

    def gibbs_sampling(self, visible_data, num_steps=1):
        for _ in range(num_steps):
            hidden_samples = self.sample_hidden_given_visible(visible_data)
            visible_samples = self.sample_visible_given_hidden(hidden_samples)
        return visible_samples
    
    def evaluate(self,y_true, y_hat):
        mse = mean_squared_error(y_true, y_hat)
        rmse = np.sqrt(mse)
        return rmse

    def train(self, data, num_epochs=1,learning_rate=0.1):
        f=[]
        for i in data:
            i=i.reshape(1,14)
            m=[]
            min_rmse=1000
            for j in range(num_epochs):
                hidden_samples = self.sample_hidden_given_visible(i)
                visible_samples = self.sample_visible_given_hidden(hidden_samples)
                
                sample_rmse=self.evaluate(i,visible_samples)
                
                if sample_rmse<min_rmse:
                    min_rmse=sample_rmse
                    min_sample=hidden_samples
                
                positive_hidden_activations = np.dot(i, self.weights) + self.hidden_bias
                positive_hidden_probs = self.sigmoid(positive_hidden_activations)
                positive_associations = np.dot(i.T, positive_hidden_probs)

                negative_hidden_activations = np.dot(visible_samples, self.weights) + self.hidden_bias
                negative_hidden_probs = self.sigmoid(negative_hidden_activations)
                negative_associations = np.dot(visible_samples.T, negative_hidden_probs)

                # Update parameters
                self.weights += learning_rate * (positive_associations - negative_associations) 
                self.visible_bias += learning_rate * np.mean(i - visible_samples, axis=0)
                self.hidden_bias += learning_rate * np.mean(positive_hidden_probs - negative_hidden_probs, axis=0)
                if j==num_epochs-1:
                    m.append(min_sample)
            f.append(m)
        return f
                

In [14]:
rbm=GaussianRBM(14,64)
new_x_train=rbm.train(rbm_train,num_epochs=100,learning_rate=0.01)
new_x_test=rbm.train(rbm_test,num_epochs=100,learning_rate=0.01)

In [15]:
new_x_train=np.array(new_x_train).reshape(20631,64)
new_x_test=np.array(new_x_test).reshape(13096,64)

In [16]:
new_cols=[f's_{i}' for i in range(1,65)]
nx_train=pd.DataFrame(new_x_train,columns=new_cols)
nx_test=pd.DataFrame(new_x_test,columns=new_cols)

In [17]:
nx_train[['unit_nr','RUL']]=X_train[['unit_nr','RUL']]
nx_test[['unit_nr','RUL']]=X_test[['unit_nr','RUL']]

In [24]:
new_test=[]
for i in range(1,101): #change it here
    df=nx_test.loc[ex_test['unit_nr']==i]
    new_test.append(df.iloc[-25:,0:64].to_numpy())
new_test=np.array(new_test).reshape(100,25,64)#change it here

In [25]:
from tensorflow.keras.preprocessing.sequence import pad_sequences as pad
def sequences(x,window=25):
    M=[]
    N=[]
    cnt=[]
    for i in range(1,101):
        sub_train=x.loc[x['unit_nr']==i]
        array=sub_train.to_numpy()
        X=[]
        Y=[]
        count=0
        label=list()
        for j in range(len(array)-window):
            row=[a for a in array[j:j+window,0:64]]
            X.append(row)
            label=array[j+window,65:66]
            Y.append(label)
            count+=1
        for j in range(len(array)-window,len(array)):
            row=[a for a in array[j:len(array),0:64]]
            X.append(row)
            m=array[len(array)-1,65:66]
            Y.append([-100])
        cnt.append(count)
        padded_sequences = pad(X, padding='post', dtype='float32',value=-100)
        M.append(padded_sequences)
        N.append(Y)
    X_sequences=pad(M, padding='post', dtype='float32',value=-100)
    Y_sequences=pad(N, padding='post', dtype='float32',value=-100)
    return np.array(X_sequences),np.array(Y_sequences),cnt

seq_x_train,seq_y_train,count1=sequences(nx_train)

In [26]:
from sklearn.model_selection import train_test_split
x1,x2,y1,y2=train_test_split(seq_x_train,seq_y_train,test_size=0.2,shuffle=True,random_state=42)
func= lambda i: i.reshape(i.shape[0]*i.shape[1],i.shape[2],i.shape[3])
x1=func(x1)
x2=func(x2)
y1=y1.flatten()
y2=y2.flatten()

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Masking,Dense,Dropout,LeakyReLU
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import R2Score
from tensorflow.keras.optimizers import Adam

model1 = Sequential()
model1.add(Masking(mask_value=-100,input_shape=(25,64)))
model1.add(Dropout(0.3))
model1.add(LSTM(128,activation='sigmoid',return_sequences=True))
model1.add(Dropout(0.3))
model1.add(LSTM(128,activation='sigmoid'))
model1.add(Dense(8,LeakyReLU(0.3)))
model1.add(Dense(1, 'linear'))
model1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_1 (Masking)             │ (None, 25, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 25, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 25, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 25, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │         1,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 231,441 (904.07 KB)

 Trainable params: 231,441 (904.07 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:

model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), metrics=[R2Score()])

In [32]:
model1.fit(x1, y1, validation_data=(x2, y2), epochs=10)

Epoch 1/10
905/905 ━━━━━━━━━━━━━━━━━━━━ 24s 23ms/step - loss: 5729.9424 - root_mean_squared_error: 75.3000 - val_loss: 2174.0024 - val_root_mean_squared_error: 46.6262
Epoch 2/10
905/905 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - loss: 1911.9077 - root_mean_squared_error: 43.6938 - val_loss: 2330.6753 - val_root_mean_squared_error: 48.2771
Epoch 3/10
905/905 ━━━━━━━━━━━━━━━━━━━━ 21s 24ms/step - loss: 1915.8964 - root_mean_squared_error: 43.6613 - val_loss: 1557.5018 - val_root_mean_squared_error: 39.4652
Epoch 4/10
905/905 ━━━━━━━━━━━━━━━━━━━━ 22s 24ms/step - loss: 1478.6515 - root_mean_squared_error: 38.4411 - val_loss: 2436.3689 - val_root_mean_squared_error: 49.3596
Epoch 5/10
905/905 ━━━━━━━━━━━━━━━━━━━━ 22s 24ms/step - loss: 2796.8035 - root_mean_squared_error: 52.7991 - val_loss: 2320.0449 - val_root_mean_squared_error: 48.1668
Epoch 6/10
905/905 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step - loss: 1958.9225 - root_mean_squared_error: 44.2250 - val_loss: 1582.9700 - val_root_mean_squared_error: 

In [33]:
result=model1.predict(new_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step


In [34]:
from sklearn.metrics import r2_score
def evaluate(y_true, y_hat, label='test'):
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    print(f'set RMSE:{rmse}')
    r2=r2_score(y_true,y_hat)
    print(r2)
    
evaluate(actual,result.round())

set RMSE:42.08883462392372
-0.025827407836914062
